In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import numpy as np
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sympy.combinatorics.permutations import Permutation
import ast
#Importing files, given_solutions is always the current optimum lists of moves.
puzzle_info = pd.read_csv('/kaggle/input/santa-2023/puzzle_info.csv')
puzzles = pd.read_csv('/kaggle/input/santa-2023/puzzles.csv')
given_solutions = pd.read_csv('/kaggle/input/wreath-100-some-improvement/submission (7).csv')
#Getting dictionaries of moves for each puzzle type.
puzzle_1_moves = puzzle_info[puzzle_info['puzzle_type'] == 'cube_2/2/2']['allowed_moves'].values[0]
puzzle_2_moves = puzzle_info[puzzle_info['puzzle_type'] == 'cube_3/3/3']['allowed_moves'].values[0]
puzzle_3_moves = puzzle_info[puzzle_info['puzzle_type'] == 'globe_1/8']['allowed_moves'].values[0]
puzzle_4_moves = puzzle_info[puzzle_info['puzzle_type'] == 'wreath_100/100']['allowed_moves'].values[0]
puzzle_1_moves = ast.literal_eval(puzzle_1_moves)
puzzle_2_moves = ast.literal_eval(puzzle_2_moves)
puzzle_4_moves = ast.literal_eval(puzzle_4_moves)
puzzle_3_moves = ast.literal_eval(puzzle_3_moves)
#We need to consider the given moves as well as their inverses, so here we add the inverses into a new dictionary.
inverse_dict_cube_2 = {}
for key, value in puzzle_1_moves.items():
    inverse_dict_cube_2[key] = Permutation(value)
    inverse_dict_cube_2['-' + key] = Permutation(value) ** -1
inverse_dict_cube_3 = {}
for key, value in puzzle_2_moves.items():
    inverse_dict_cube_3[key] = Permutation(value)
    inverse_dict_cube_3['-' + key] = Permutation(value) ** -1
inverse_dict_wreath_100 = {}
for key, value in puzzle_4_moves.items():
    inverse_dict_wreath_100[key] = Permutation(value)
    inverse_dict_wreath_100['-' + key] = Permutation(value) ** -1
inverse_dict_globe_1_8 = {}
for key, value in puzzle_3_moves.items():
    inverse_dict_globe_1_8[key] = Permutation(value)
    inverse_dict_globe_1_8['-' + key] = Permutation(value) ** -1
print(puzzles['puzzle_type'].value_counts())

def generatepaths(length, words, dictionary,perm, value):
    '''given a length and a dictionary corresponding moves to their permutation, returns a dictionary of all permutations that can 
        be achieved with n or less moves and their corresponding set of moves
        inputs: length-the length of words that you want to consider
                words- used for recursion, should be initialized as an empty dictionary
                dictionary- a dictionary corresponding moves to the permutations they represent
                perm- used for recursion, should be initialized as the trivial permutation
                value- used for recuresion, should be initialized as an empty word
        returns: a dictionary corresponding sequences of moves and the permutation they cause.'''
    if value.count('.') >= length:
        return words
    else:
        for i in dictionary.keys():
            oldperm = perm
            perm = dictionary[i] * perm
            oldvalue = value
            value = value + i + '.'
            if perm in words:
                return words
            else:
                words[perm] = value
                words = generatepaths(length, words, dictionary, perm, value)
            perm = oldperm
            value = oldvalue
    return words
#step is part of an alternate way to generate sequences of moves and their permutations. It is more efficient than the old generate paths
#becuase it does not consider words which are longer than they need to be.
def step(words, dictionary):
    '''Given a dictionary corresponding sequences of moves and their permutations, adds all possible moves to the end of the sequences and produces a new dictionary.
        This function will generally not be directly used, and should instead be used primarily through the steps function.
        inputs- words: the previous dictionary with sequences of moves
        dictionary- the dictionary with each move and its permutation
        returns- a dictionary with all sequences of moves with length one greater than the initial dictionary and their permutations.'''
    newwords = words.copy()
    for word in words:
        for step in dictionary:
            perm = dictionary[step] * word
            if not perm in words:
                newwords[perm] = words[word] + step + '.'
    return newwords
def randomstep(words, dictionary, percent, allwords):
    '''Similar to step, except instead of adding all possible moves, chooses a random set of possible words. Should only be called indirectly through randomsteps
        inputs- words: previous dictionary
                dictionary: dictionary corresponding moves to their permutation
                percent: the percent of moves out of the total possible moves that you want to consider at each step
                allwords: list of all words generated up to this point in the process
        returns- dictionary of all words of length one greater than its input words and their corresponding permutations'''
    number = int(np.round(len(list(dictionary)) * percent))
    newwords = {}
    for word in words:
        steps = random.sample(list(dictionary), number)
        for step in steps:
            perm = dictionary[step] * word
            if not perm in allwords:
                newwords[perm] = words[word] + step + '.'
    return newwords
def steps(size, dictionary, steps):
    '''Runs step several times to generate a dictionary of all words of less than or equal to a certain lengths and their permutations
        inputs- size: the size of the set being permutated by the permuations.
                dictionary: dictionary of all moves and their corresponding permutations
                steps: number of steps to run.
                returns: a dictionary of all permutations that can be achieved by a steps or fewer moves and the sequence of moves that achieves that permutation'''
    words = {Permutation(size - 1): ''}
    for i in range(steps):
        words = step(words, dictionary)
    return(words)
def randomsteps(size, dictionary, steps, percent):
    '''Similar to steps but instead of generating all permutations, generates a random subsample'''
    words = {Permutation(size - 1): ''}
    newwords = {Permutation(size - 1): ''}
    for i in range(steps):
        newwords = randomstep(newwords, dictionary, percent, words)
        words.update(newwords)
    return words
#generating some lists of sequences of moves
possibles_cube_2 = steps(24, inverse_dict_cube_2, 5)
possibles_cube_3 = randomsteps(54, inverse_dict_cube_3, 5, 0.3)
possibles_wreath_100 = steps(198, inverse_dict_wreath_100, 5)
possibles_globe_1_8 = steps(32, inverse_dict_globe_1_8, 3)
            

def getstates(word,dictionary,initstate):
    '''given a sequence of moves and an initial state, generates a dictionary of all states moved through as the moves are successively applied
            inputs- word: the sequence of moves
                dictionary: a dictionary of possible moves and the permutations they achieve
                initstate: the initial permutation of the set
            returns- a dictionary of all states moved through by successively applying the permutations in the word'''
    moves = word.split('.')
    initstate = initstate.split(';')
    states = {0:initstate}
    for i in range(len(moves)):
        initstate = dictionary[moves[i]](initstate)
        states[i + 1] = initstate
    return states


def checkforshortcuts(length, word, dictionary,initstate, possibles):
    '''given a word and a dictionary of sequences moves, tries to find a shorter word representing the same permutation
        length: the maximum length of words in words
        word: the word the function attempts to shorten
        dictionary: a dictionary of moves and the permutations they represent
        initstate: the initial state of the set being permuted
        possibles: a list of words that can be used to shorten word
        returns- If possible, a shorter word representing the same permuation, otherwise returns the original word'''
    array = word.split('.')
    states = getstates(word, dictionary, initstate)
    for i in range(len(array) - length):
        sublength = i + length + 1
        #sublength represents the length of the subword we are trying to replace.
        for j in range(len(array) - sublength + 1):
            initstate = states[j]
            finstate = states[j+sublength]
            for key, value in possibles.items():
                if key(initstate) == finstate:
                    newstring = ''
                    for a in range(j):
                        newstring = newstring + array[a] + '.'
                    newstring = newstring + value
                    for b in range(len(array) - j - sublength):
                        newstring = newstring + array[j + sublength + b] + '.'
                    return newstring[0:len(newstring)-1]
    return word

def reduce(word, length, dictionary, initstate, runs, possibles):
    '''given a word and a dictionary of possible sequences of moves, repeatedly shortens the word without changing the permutaiton it represents
            inputs- word: the word to be shortened
                length- the maximum length of a word in possibles
                dictionary- a dictionary of moves and the permutations they represent
                initstate- the initial state of the set being permuted
                runs- the number of times to try to shorten the word
                possibles- a list of possible sequences of words to shorten the word
            returns- a shortened version of the word if possible, otherwise the original word
            '''
    for i in range(runs):
        print('run ')
        print(i)
        print(word.count('.'))
        newstring = checkforshortcuts(length, word,dictionary,initstate, possibles)
        if word.count('.') == newstring.count('.'):
            break
        else:
            word = newstring
    print(newstring.count('.'))
    return newstring
def checkanswer(word, initial_state, goal_state, dictionary):
    '''given a word, an initial state, and a goal state, decides if the permutation the word represents transforms the initial state to the goal state. Useful for troubleshooting.
        inputs-word: the word being checked
                initial_state: the initial state of the word being permuted
                goal_state: the solved state of the puzzle
                dictionary: a dictionary corresponding moves to the permutation they represent'''
    moves = word.split('.')
    if type(initial_state) == str:
        state = initial_state.split(';')
    else:
        state = initial_state
    for move in moves:
        state = dictionary[move](state)
    if type(goal_state) == str:
        goal_state = goal_state.split(';')
    if state == goal_state:
        return True
    else:
        return False
#finally, we apply the functions to reduce the length of words solving the puzzles piece by piece. 
solutions = []
for i in range(338):
    solutions.append(given_solutions.iloc[i]['moves'])
for i in range(10):
    print('solving puzzle ')
    print(i + 338)
    solution = reduce(given_solutions.iloc[i + 338]['moves'], 5, inverse_dict_globe_1_8, puzzles.iloc[i + 338]['initial_state'], 10, possibles_globe_1_8)
    solutions.append(solution)
for i in range(50):
    solutions.append(given_solutions.iloc[i + 338]['moves'])
ids = []
for i in range(398):
    ids.append(i)
dictionary = {'id':ids, 'moves': solutions}
solutions_df = pd.DataFrame(dictionary)
display(solutions_df)
solutions_df.to_csv('submission.csv', index = False)

/kaggle/input/wreath-1212-improvement/submission (5).csv
/kaggle/input/sumbission-9/submission (9).csv
/kaggle/input/improved-big-wreaths/submission (4).csv
/kaggle/input/submission-7/submission (7).csv
/kaggle/input/santa-2023/sample_submission.csv
/kaggle/input/santa-2023/puzzles.csv
/kaggle/input/santa-2023/puzzle_info.csv
/kaggle/input/working-on-big-wreath/submission (6).csv
/kaggle/input/first-half-big-wreaths/submission (3).csv
/kaggle/input/submission-10/submission (10).csv
/kaggle/input/wreath-100-some-improvement/submission (7).csv
/kaggle/input/submission-8/submission (8).csv
puzzle_type
cube_3/3/3        120
cube_4/4/4         60
cube_5/5/5         35
cube_2/2/2         30
wreath_6/6         20
globe_3/4          15
wreath_7/7         15
cube_6/6/6         12
globe_1/8          10
wreath_12/12       10
globe_3/33          8
cube_10/10/10       5
cube_9/9/9          5
wreath_21/21        5
cube_8/8/8          5
globe_1/16          5
globe_2/6           5
cube_7/7/7          

,id,moves
0,0,r1.-f1
1,1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,3,-f0.-r0.-f0.-d0.-f1.-f1.-d1.-f1.-r0.-f1.-d0.-f...
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...,...
393,393,-f12.f10.f14.-r0.-f10.-r4.-f9.f12.f2.f13.f18.-...
394,394,f9.r1.f17.r6.f16.f19.f2.-f6.-f8.f3.-r4.f15.-r4...
395,395,r3.f17.f13.f1.f10.-f14.f7.f7.-f9.-r1.f0.-f8.r4...
396,396,r5.-f4.-f5.r6.f4.f8.f13.-r1.r2.f11.-f17.-f5.-r...
